In [7]:
import bats
import numpy as np
import time

In [4]:
n = 100
X = np.random.normal(size=(n,2))

In [5]:
bats.L1Dist()

In [11]:
def time_BATS_update_L1(X):
    tlist = []
    
    m1 = bats.Euclidean()
    m2 = bats.L1Dist()
    
    data = bats.DataSet(bats.Matrix(X)) # put into a bats.DataSet
    
    t0 = time.monotonic()
    F_X = bats.RipsFiltration(data, m1, np.inf, 2) # generate a RipsFiltration
    R_X = bats.reduce(F_X, bats.F2()) # reduce with F2 coefficients
    t1 = time.monotonic()
    tlist.append(t1-t0)
    #print("time to reduce: {} sec.".format(t1 - t0))
    
    t0 = time.monotonic()
    F_Y = bats.RipsFiltration(data, m2, np.inf, 2) # generate a new RipsFiltration
    UI = bats.UpdateInfoFiltration(F_X, F_Y)
    R_X.update_filtration_general(UI)
    t1 = time.monotonic()
    #print("time to update: {} sec.".format(t1 - t0))
    tlist.append(t1-t0)
    
    return tlist


In [12]:
record = []
for i in range(30):
    record.append(time_BATS_update_L1(X))

In [13]:
# get mean time
record = np.array(record)
np.mean(record, axis = 0)

array([0.59278008, 0.37799254])

# Comparison on different packages

In [22]:
import numpy as np
from ripser import Rips

t0 = time.monotonic()
rips = Rips(maxdim=2, metric = 'cosine')
diagrams = rips.fit_transform(X)
t1 = time.monotonic()
print("time to compute: {} sec.".format(t1 - t0))

TypeError: __init__() got an unexpected keyword argument 'metric'

In [24]:
from ripser import ripser

dgms = ripser(X)['dgms']

In [26]:
dgms.type

AttributeError: 'list' object has no attribute 'type'

In [20]:
import gudhi as gd
t0 = time.monotonic()
rips_complex = gd.RipsComplex(points=X, max_edge_length=np.inf)
simplex_tree = rips_complex.create_simplex_tree(max_dimension=2)
diag = simplex_tree.persistence()
t1 = time.monotonic()
print("compute1: {} sec.".format(t1 - t0))

compute1: 0.11691536299986183 sec.


In [21]:
t0 = time.monotonic()
m1 = bats.Euclidean()
data = bats.DataSet(bats.Matrix(X)) # put into a bats.DataSet
F_X = bats.RipsFiltration(data, m1, np.inf, 2) # generate a RipsFiltration
R_X = bats.reduce(F_X, bats.F2()) # reduce with F2 coefficients
t1 = time.monotonic()
print("compute1: {} sec.".format(t1 - t0))

compute1: 0.6973471460000837 sec.
